### Importações necessárias

In [2]:
import numpy as np
import math
from numpy import *
from numpy.linalg import inv
import numpy.linalg as la
import csv

# Parêmtros de entrada

In [3]:
str_nome = "blood"
str_nome_base = "bloodPressureNishisatobook-base.D18.N15.dat"
str_nome_regras = "Blood_regras.txt"

 # Cálculo Dual Scaling

#### Ler arquivo base e criar Matriz padrão respota

In [14]:
arq = open(str_nome_base, 'r')
texto = arq.readlines()
base = []
item = set()
for linha in texto:
    valores_linha = []
    for value in linha.split():
        valores_linha.append(int(value))
        item.add(int(value))
    base.append(valores_linha)   
arq.close()

#print(valores)

F = np.zeros(shape=(len(base), len(item)))
for i in range(len(base)):
    for value in base[i]:
        F[i,value-1] = 1

#print(F)

#### Número de Dimensões do Espaço-solução (ns)

In [15]:
q = len(base[0]) #número de questões (categorias)
m = F.shape[1] #número de itens
n = F.shape[0] #número de transações

ns = m - q - 1

print('Total de Dimensões: ', ns,'.')

Total de Dimensões:  11 .


#### Matriz Diagonal de Linhas (Dr) 

In [16]:
#Cálculo da Matriz Diagonal de Linhas de F
fr = np.sum(F, axis=1)
Dr = np.diagflat(fr)

#print(fr)

#Cálculo da Matriz Diagonal de Colunas de F
fc = np.sum(F, axis=0)
Dc = np.diagflat(fc)
#print(fc)

#### Matriz Resultante (M)

In [17]:
Ft = np.transpose(F) #Matriz Transporta de F
Dri = np.linalg.inv(Dr) #Matriz Invertida de Dr
Dci = np.linalg.inv(Dc) #Matriz Invertida de Dc

#Cálculo da Matriz M
M = np.dot(np.dot(np.dot(Ft,Dri),F), Dci)
#print(M)

#### Autovalores (AD) e Autovetores (AV) de M

In [19]:
#Array de Autovalores (uAD) e Autovetores (uAV) de M.
uAD, uAV = np.linalg.eig(M)

#Ordenando os Autovalores
AD = np.flip(np.sort(uAD, axis=0),0)

#np.savetxt('AV_python.txt',uAV.astype(np.float32))

#Ordenando os Autovetores em função dos Autovalores
uAVt = uAV.T.tolist()
AV=[]
for r in AD:
    #print(n)
    ix = uAD.tolist().index(r)
    AV.append(uAVt[ix])

#convertendo o resultado de um tipo complexo para Float32    
AD = AD.astype(np.float32)
AV = np.array(AV).T.astype(np.float32)

#removendo a primeira dimensão
AD = np.delete(AD,0,0)
AV = np.delete(AV,0,1)

#removendo as demais dimensões que devem ser desconsideradas (de forma que fiquem com dimensão ns)
count = ns
while count < AD.shape[0]:
    AD = np.delete(AD,count,0)
    AV = np.delete(AV,count,1)

#print(AD)
#print(AV)

C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: ComplexWarning: Casting complex values to real discards the imaginary part


#### rho

In [20]:
# Cálculo da Matriz unidimensional rho
rho = []

for i in range(ns):
    rho.append(math.sqrt(AD.tolist()[i]))

rho = np.array(rho)
#print(rho)   

#### Matriz Produto de Hadamard (H)

In [21]:
#Calculo da produto de Hadamard de Autovetores * autovetores
H = np.multiply(AV,AV)
#print(H.astype(np.float32))

#### Matriz T

In [23]:
#Matriz resultante T
T = np.dot(Dc, H)

#print(T.astype(np.float32))

#### Matriz Cc

In [40]:
sumCols = np.sum(T, axis=0).tolist()
#print (sumCols)
ft = np.sum(F)
# print(ft)
# print(sumCols)
Cc = []
for i in range(ns):
    r = math.sqrt(ft/float(sumCols[i]))
    Cc.append(r)

# print (Cc) 

#### Peso padrão dos Itens (x-normed)

In [41]:
#multiplicação da matriz de autovetores pela matrix Cc
x_normed = []
for r in AV:
    x_normed.append(np.multiply(r,Cc))

#print (np.array(x_normed))

#### Peso projetado dos Itens (x-projected)

In [42]:
#Resultado Principal do Dual Scaling
x_projected = []
for r in x_normed:
    x_projected.append(np.multiply(r,rho))

x_projected = np.array(x_projected)
print (x_projected.shape)

(18, 11)


#### Calculando Delta (d)

In [24]:
d = []
s = 100/np.sum(AD)
for r in AD:
    d.append(s*r)
#print(d)

# Calculo de Distância

## Distância entre itens (matriz de distância)

In [44]:
# não precisa calcular
DM = np.ndarray(shape=(x_projected.shape[0],x_projected.shape[0]), dtype=float)
for i in range(x_projected.shape[0]):
    for j in range(x_projected.shape[0]):
        quad_dist = 0
        for aux in range(ns):
            dist = (x_projected.item((i, aux)) / math.sqrt(fc.item(i)/n)) - (x_projected.item((j, aux)) / math.sqrt(fc.item(j)/n))
            quad_dist += (dist * dist)*rho.item(aux)
        DM.itemset((i,j),quad_dist)
        DM.itemset((j,i),quad_dist)

# print(DM)

## Função ponto médio

In [45]:
def ponto_medio(ids_item, x_projected, ns):
    ptm = np.zeros(ns)
    for item in ids_item:
        for i in range(x_projected.shape[1]):
            ptm[i] = ptm[i] + x_projected[item-1][i]
    ptm = ptm / float(len(ids_item))
    return ptm

## Função de cálculo de distância de antecedente e consequente

In [46]:
def distancia_antecedente_consequente(ptmA, supS, ptmC, supC, rho):
    quad_dist = 0
    for aux in range(ns):
        dist = ( ptmA[aux] / math.sqrt(supS) ) - (ptmC[aux] / math.sqrt(supC))
        quad_dist += (dist * dist)*rho[aux]
    return quad_dist    

## Função de cálculo de distância do ponto até a origem

In [47]:
def distancia_ponto_origem(pt, sup, rho):
    quad_dist = 0
    for aux in range(ns):
        dist = pt[aux] / math.sqrt(sup)
        quad_dist += (dist * dist)*rho[aux]
    return quad_dist

# Métricas de Interesse

## confiança P(B | A) = AUB / A

In [48]:
def get_confianca(sup, supA):
    return sup/supA

## lift P(AB)/P(A)P(B)

In [49]:
def get_lift(sup, supA, supB):
    return sup/supA*supB

## all-confidence 

In [50]:
def get_allconfidence(sup, ant, conseq, fc, n):
    maiorsuporte = 0
    for item in ant:
        if maiorsuporte < fc[item-1]:
            maiorsuporte = fc[item-1]
    for item in conseq:
        if maiorsuporte < fc[item-1]:
            maiorsuporte = fc[item-1]
    
    maiorsuporte = maiorsuporte/n
    
    return sup/maiorsuporte

# Preparando metadado

In [51]:
arq_regras = open(str_nome_regras, 'r')
arq_metadado = open(str_nome + ".csv", 'w', encoding='utf-8', newline='')

escreve_csv = csv.writer(arq_metadado, delimiter=';')
escreve_csv.writerow(['Antecedente', 'Consequente', 'Dist. Anteced. x Origem', 'Dist. Conseq. x Origem',
                      'Dist. Anteced. -> Conseq.', "Dist. Anteced. - Conseq.", "Suporte Ant.", "Suporte Cons.",
                      "Suporte", "Confianca", "Lift", "all-confianca"])

for linha_regra in arq_regras:
    info = linha_regra.split()
    antecedente = [int(i) for i in info[0].split(",")]
    consequente = [int(i) for i in info[1].split(",")]
    
    #métricas
    suporte = float(info[2])
    confianca = float(info[3])
    suporteA = float(info[4])
    suporteC = float(info[5])
    lift = get_lift(suporte,suporteA,suporteC)
    allconfianca = get_allconfidence(suporte, antecedente, consequente, fc, n)

    # distâncias
    ptmA = ponto_medio(antecedente, x_projected, ns)
    ptmC = ponto_medio(consequente, x_projected, ns)
    
    dist_ant_orig = distancia_ponto_origem(ptmA, suporteA, rho)
    dist_conseq_orig = distancia_ponto_origem(ptmC, suporteA, rho)
    dist_ant_conseq = distancia_antecedente_consequente(ptmA, suporteA, ptmC, suporteC, rho)
    dist_ant_menos_conseq = dist_ant_orig - dist_conseq_orig
    
    #escreve arquivo csv
    b = [antecedente, consequente, dist_ant_orig, dist_conseq_orig, dist_ant_conseq, dist_ant_menos_conseq, 
         suporteA, suporteC, suporte, confianca, lift, allconfianca]
    escreve_csv.writerow(b)
    
arq_regras.close()
arq_metadado.close()